In [1]:
from tvb.simulator.lab import *
from toolbox import timeseriesPlot, FFTplot
import numpy as np

With **Paupau_ connectivity**, 4 nodes and just 1 long range connection: *[iA1(blue) -> iA2(red)] = 1*.

In [2]:
simLength = 5000 # ms - relatively long simulation to be able to check for power distribution
samplingFreq = 1000 #Hz

integrator = integrators.HeunDeterministic(dt=1000/samplingFreq)

conn = connectivity.Connectivity.from_file("paupau_.zip")
mon = (monitors.Raw(),)

WARNING  File 'hemispheres' not found in ZIP.


# CHOOSE MODEL

In [3]:
m = models.Generic2dOscillator(I=np.array([4]))
coup = coupling.Linear(a=np.array([0.02]), b=np.array([0]))
transient=500

In [31]:
m = models.WilsonCowan(P=np.array([0.51]), Q=np.array([0]), # Original P at 0.31
                       a_e=np.array([4]), a_i=np.array([4]),
                       alpha_e=np.array([1]), alpha_i=np.array([1]),
                       b_e=np.array([1]), b_i=np.array([1]),
                       c_e=np.array([1]), c_ee=np.array([3.25]), c_ei=np.array([2.5]),
                       c_i=np.array([1]), c_ie=np.array([3.75]), c_ii=np.array([0]),
                       k_e=np.array([1]), k_i=np.array([1]),
                       r_e=np.array([0]), r_i=np.array([0]),
                       tau_e=np.array([10]), tau_i=np.array([20]),
                       theta_e=np.array([0]), theta_i=np.array([0]))

coup = coupling.Linear(a=np.array([0.1]))
conn.speed = np.array([3])
transient=500

In [33]:
m = models.JansenRit(A=np.array([3.25]), B=np.array([22]), J=np.array([1]), # Original mu at 0.09
                     a=np.array([0.1]), a_1=np.array([135]), a_2=np.array([108]),
                     a_3=np.array([33.75]), a_4=np.array([33.75]), b=np.array([0.05]),
                     mu=np.array([0.19]), nu_max=np.array([0.0025]), p_max=np.array([0]), p_min=np.array([0]),
                     r=np.array([0.56]), v0=np.array([6]))

coup = coupling.SigmoidalJansenRit(a=np.array([1]), cmax=np.array([0.005]),  midpoint=np.array([6]), r=np.array([0.56]))
conn.speed=np.array([6])
transient=500

#### Test the model

In [34]:
# Run simulation
sim = simulator.Simulator(model=m, connectivity=conn, coupling=coup,  integrator=integrator, monitors=mon)
sim.configure()
output = sim.run(simulation_length=simLength)
# Extract data cutting initial transient
raw_data = output[0][1][transient:, 0, :, 0].T
raw_time = output[0][0][transient:]
regionLabels = conn.region_labels
regionLabels=list(regionLabels)

# Check initial transient and cut data
timeseriesPlot(raw_data, raw_time, regionLabels, mode="inline")

# Fourier Analysis plot
FFTplot(raw_data, simLength, regionLabels,  mode="inline")

c:\users\jesca\pycharmprojects\brainmodels\venv\lib\site-packages\tvb\simulator\models\jansen_rit.py:273: RuntimeWarning:

overflow encountered in _numba_dfun_jr

c:\users\jesca\pycharmprojects\brainmodels\venv\lib\site-packages\tvb\simulator\coupling.py:365: RuntimeWarning:

overflow encountered in exp



# CHOOSE STIMULUS

In [ ]:
eqn_t = equations.PulseTrain()
eqn_t.parameters['onset'] = 1535.0 # ms Stimulation from onset to end
eqn_t.parameters['T'] = 100.0 # ms Period of stimulation: time from stimulation[0] init to stimulation[1] init.
eqn_t.parameters['tau'] = 50.0 # ms length of stimulation pulse

In [13]:
eqn_t = equations.Sinusoid()
eqn_t.parameters['amp'] = 0.25
eqn_t.parameters['frequency'] = 9 #Hz
eqn_t.parameters['onset'] = 2000 #ms
eqn_t.parameters['offset'] = 3000 #ms

In [27]:
eqn_t = equations.DriftSinusoid()
eqn_t.parameters['amp'] = 0.1
eqn_t.parameters['f_init'] = 10 #Hz
eqn_t.parameters['f_end'] = 20 #Hz
eqn_t.parameters['onset'] = 1000 #ms
eqn_t.parameters['offset'] = 4000 #ms
eqn_t.parameters['feedback'] = True # Grow&Shrink (True) - Grow|Shrink(False)
eqn_t.parameters['sim_length'] = simLength # ms
eqn_t.parameters['dt'] = 1000/samplingFreq # in ms

In [28]:
weighting = np.zeros((4, ))
weighting[[0]] = 0.01

In [29]:
stimulus = patterns.StimuliRegion(
    temporal=eqn_t,
    connectivity=conn,
    weight=weighting)


stimulus.configure_space()
stimulus.configure_time(np.arange(0, simLength, 1))

In [30]:
sim = simulator.Simulator(model=m, connectivity=conn, coupling=coup,  integrator=integrator, monitors=mon, stimulus=stimulus)
sim.configure()
output = sim.run(simulation_length=simLength)
# Extract data cutting initial transient
raw_data = output[0][1][transient:, 0, :, 0].T
raw_time = output[0][0][transient:]
regionLabels = conn.region_labels
regionLabels=list(regionLabels)
regionLabels.insert(5,"stimulus")

# average signals to obtain mean signal frequency peak
data = np.concatenate((raw_data, stimulus.temporal_pattern[:,transient:]), axis=0)
# Check initial transient and cut data
timeseriesPlot(data, raw_time, regionLabels, mode="inline")
# Fourier Analysis plot
FFTplot(raw_data, simLength, regionLabels,  mode="inline")